In [1]:
!pip install mne joblib pycatch22 tqdm tqdm_joblib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import glob
import numpy as np
import mne
import xml.etree.ElementTree as ET
from scipy.signal import butter, iirnotch, iirnotch, sosfiltfilt
from joblib import Parallel, delayed
from pycatch22 import catch22_all
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib

# Configuration
BASE_DIR = '/content/drive/MyDrive/4830_project/data/mesa_data/'
EDF_DIR = os.path.join(BASE_DIR, 'mesa_200_subset_edfs')
ANNOTATION_DIR = os.path.join(BASE_DIR, 'mesa_200_annotations-events-nssr')
OUTPUT_DIR = '/content/drive/MyDrive/4830_project/MESA_data/processed_MESA'
os.makedirs(OUTPUT_DIR, exist_ok=True)


CHANNELS_TO_LOAD = ["EEG1", "EOG-L", "EOG-R"]
TARGET_SFREQ = 256.0
LOW_FREQ = 0.5
HIGH_FREQ = 30.0
EPOCH_LENGTH = 30.0
SEQ_LENGTH = 20
SEQ_STRIDE = 10

# Sleep stage mapping based on NSRR annotations
ANNOTATION_MAP = {
    "0": 0,  # Wake
    "1": 1,  # N1
    "2": 2,  # N2
    "3": 3,  # N3
    "5": 4   # REM
}

def design_notch_filter(fs, freq=60.0, quality=30.0):
    """IIR notch filter, return it in second-order sections."""
    w0 = freq / (0.5 * fs)
    b, a = iirnotch(w0, quality)
    sos = butter(2, [freq-1, freq+1], btype='bandstop', fs=fs, output='sos')
    return sos

def apply_notch_filter(signal, sos):
    """Apply the pre-designed notch filter."""
    return sosfiltfilt(sos, signal)

def parse_xml_annotations(xml_path):
    """Parse the XML file to extract sleep stage annotations."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    stages = []
    for event in root.findall(".//ScoredEvent"):
        event_type = event.findtext("EventType", default="")
        if event_type != "Stages|Stages":
            continue
        concept = event.findtext("EventConcept", default="").strip().split("|")[-1]  # Get the last part of the stage name
        if concept not in ANNOTATION_MAP:
            continue

        start = float(event.findtext("Start", 0))
        duration = float(event.findtext("Duration", 0))
        stage_id = ANNOTATION_MAP[concept]

        # Split duration into multiple 30-second epochs
        num_epochs = int(duration // 30)
        for i in range(num_epochs):
            epoch_start = start + i * 30
            stages.append((epoch_start, 30.0, stage_id))
    if len(stages) == 0:
        raise ValueError(f"No valid stages found in {xml_path}")

    return stages

def extract_catch22_features(epoch_data):
    """Compute CATCH22 features for each epoch and channel."""
    n_epochs, n_channels, n_samples = epoch_data.shape
    features = []
    for i in range(n_epochs):
        epoch_feats = Parallel(n_jobs=-1)(  # Use all cores inside a single file
            delayed(catch22_all)(epoch_data[i, ch, :]) for ch in range(n_channels)
        )
        # Only keep 'values' part from each channel
        epoch_feats = [feats['values'] for feats in epoch_feats]
        features.append(epoch_feats)
    return np.array(features)  # shape: (n_epochs, n_channels, 22)

def process_record(psg_path, xml_path, channels, target_sfreq, low_freq, high_freq, epoch_length):
    """Process a single PSG file with its corresponding XML annotations."""
    raw = mne.io.read_raw_edf(psg_path, preload=True, verbose=False)
    raw.pick_channels(channels) #ONLY KEEP OUR DESIRED CHANNELS
    if raw.info['sfreq'] != target_sfreq:
        raw.resample(target_sfreq, npad="auto", verbose=False)

    raw.filter(l_freq=low_freq, h_freq=high_freq, verbose=False)

    # Apply notch
    sos = design_notch_filter(raw.info['sfreq'], freq=60.0)

    for ch_idx in range(len(raw.ch_names)):
        signal = raw.get_data(picks=[ch_idx])[0]
        raw._data[ch_idx] = apply_notch_filter(signal, sos)

    annotations = parse_xml_annotations(xml_path)

    # Convert annotations to MNE-compatible events
    events = []
    for start, duration, stage_id in annotations:
        sample = int(start * raw.info['sfreq'])
        events.append([sample, 0, stage_id])

    events = np.array(events)
    tmin = 0.0
    tmax = epoch_length - 1 / raw.info['sfreq']
    epochs = mne.Epochs(raw, events=events, event_id=None, tmin=tmin, tmax=tmax,
                        baseline=None, preload=True, verbose=False)

    data = epochs.get_data()
    labels = epochs.events[:, -1]

    for ch in range(data.shape[1]):
        m = np.mean(data[:, ch, :])
        s = np.std(data[:, ch, :]) if np.std(data[:, ch, :]) != 0 else 1.0
        data[:, ch, :] = (data[:, ch, :] - m) / s

    return data, labels, raw.ch_names

def create_sequences(data, labels, seq_length, seq_stride):
    """Create sequences of epochs."""
    n_epochs = data.shape[0]
    sequences, seq_labels = [], []
    for start in range(0, n_epochs - seq_length + 1, seq_stride):
        sequences.append(data[start:start+seq_length])
        seq_labels.append(labels[start:start+seq_length])
    return np.array(sequences), np.array(seq_labels)

def process_and_save(psg_file, output_dir, channels):
    """Process and save data for a single PSG file."""
    try:
        rec_id = os.path.basename(psg_file).replace(".edf", "")
        tqdm.write(f"Starting {rec_id}")
        xml_file = os.path.join(ANNOTATION_DIR, os.path.basename(psg_file).replace(".edf", "-nsrr.xml"))
        if not os.path.exists(xml_file):
            tqdm.write(f"Annotation file not found for {rec_id}, skipping.")
            return
        #loading and prepro
        try:
            tqdm.write(f"Loading and preprocessing {rec_id}")
            data, labels, ch_names = process_record(psg_file, xml_file, channels,
                                                    TARGET_SFREQ, LOW_FREQ, HIGH_FREQ, EPOCH_LENGTH)
        except Exception as e:
            tqdm.write(f"Failed during process_record for {os.path.basename(psg_file)}: {e}")
            return
        #Feature extraction
        try:
            tqdm.write(f"Extracting CATCH22 for {rec_id}")
            # Before feature extraction, downsample data:
            data_downsample = mne.filter.resample(data, down=2.56, npad="auto")
            features = extract_catch22_features(data_downsample)
        except Exception as e:
            tqdm.write(f"Failed during feature extraction for {os.path.basename(psg_file)}: {e}")
            return



        # Try saving
        try:
            tqdm.write(f"Saving for {rec_id}")
            np.savez_compressed(os.path.join(output_dir, f"{rec_id}_epochs.npz"),
                                data=data.astype('float32'), labels=labels.astype('int8'), features=features.astype('float32'))
            sequences, seq_labels = create_sequences(data, labels, SEQ_LENGTH, SEQ_STRIDE)
            np.savez_compressed(os.path.join(output_dir, f"{rec_id}_sequences.npz"),
                                sequences=sequences.astype('float32'), seq_labels=seq_labels.astype('int8'))
        except Exception as e:
            tqdm.write(f"Failed during saving for {os.path.basename(psg_file)}: {e}")
            return

        tqdm.write(f"Finished {rec_id}: {data.shape[0]} epochs, {sequences.shape[0]} sequences, channels: {ch_names}")

    except Exception as e:
        tqdm.write(f"Unexpected error with {os.path.basename(psg_file)}: {e}")

In [ ]:
def main():
    psg_files = glob.glob(os.path.join(EDF_DIR, "*.edf"))
    print(f"Found {len(psg_files)} PSG files.")

    with tqdm_joblib(tqdm(desc="Processing PSGs", ncols=100, total=len(psg_files))):
        Parallel(n_jobs=1)(
            delayed(process_and_save)(f, OUTPUT_DIR, CHANNELS_TO_LOAD)
            for f in psg_files
        )

if __name__ == '__main__':
    main()

Found 200 PSG files.



Processing PSGs:   0%|                                                      | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]


Processing PSGs:   0%|                                                       | 0/25 [48:44<?, ?it/s]

Processing PSGs:   0%|                                                      | 0/200 [00:00<?, ?it/s]

Starting mesa-sleep-0001
Loading and preprocessing mesa-sleep-0001
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

Extracting CATCH22 for mesa-sleep-0193




Processing PSGs:   0%|                                                    | 0/200 [3:47:12<?, ?it/s]

Saving for mesa-sleep-0193




Processing PSGs:   0%|                                                    | 0/200 [3:47:37<?, ?it/s]

Processing PSGs:   0%|                                                    | 0/200 [3:47:37<?, ?it/s]

Processing PSGs:   0%|                                                    | 0/200 [3:47:37<?, ?it/s]

Finished mesa-sleep-0193: 1320 epochs, 131 sequences, channels: ['EEG1', 'EOG-L', 'EOG-R']
Starting mesa-sleep-0194
Loading and preprocessing mesa-sleep-0194


In [ ]:
from google.colab import runtime
runtime.unassign()